# Aufgabe 2 - Tiefe Einblicke ins Institut

Den Code immer nachvollziehbar kommentieren! Bitte beachtet, dass das Notebook von Anfang bis Ende ohne Fehler durchlaufen muss und dass die requirements.txt Datei aktualisiert wird. 

## Teilaufgabe a): Trainings-und Testdatenset

In [ ]:
from PIL import Image
import numpy as np

training_path_list = []  # ["./pfad/zu/bildA.png","./pfad/zu/bildB.png",...]
test_path_list = []  # ["./pfad/zu/bildC.png","./pfad/zu/bildD.png",...]

def training_list():
    for i in range(20):
        training_path_list.append(f"./Bilder/Flur/Flur{i+1}.jpg")
    for i in range(20):
        training_path_list.append(f"./Bilder/Labor/Labor{i+1}.jpg")
    for i in range(20):
        training_path_list.append(f"./Bilder/Professorkueche/Professorkueche{i+1}.jpg")
    for i in range(20):
        training_path_list.append(f"./Bilder/Teekueche/Teekueche{i+1}.jpg")

def test_list():
    for i in range(6):
        test_path_list.append(f"./Bilder/Flur/Flur{i+20}.jpg")
    for i in range(6):
        test_path_list.append(f"./Bilder/Labor/Labor{i+20}.jpg")
    for i in range(6):
        test_path_list.append(f"./Bilder/Professorkueche/Professorkueche{i+20}.jpg")
    for i in range(6):
        test_path_list.append(f"./Bilder/Teekueche/Teekueche{i+20}.jpg")

training_list()
test_list()
print(test_path_list)

try:
    image = Image.open(training_path_list[0])
except:
    raise ValueError("Ihr müsst einmal dieses Lageplan.png bei /usr/lageplan.png einfügen")

    

['./Bilder/Flur/Flur20.jpg', './Bilder/Flur/Flur21.jpg', './Bilder/Flur/Flur22.jpg', './Bilder/Flur/Flur23.jpg', './Bilder/Flur/Flur24.jpg', './Bilder/Flur/Flur25.jpg', './Bilder/Labor/Labor20.jpg', './Bilder/Labor/Labor21.jpg', './Bilder/Labor/Labor22.jpg', './Bilder/Labor/Labor23.jpg', './Bilder/Labor/Labor24.jpg', './Bilder/Labor/Labor25.jpg', './Bilder/Professorkueche/Professorkueche20.jpg', './Bilder/Professorkueche/Professorkueche21.jpg', './Bilder/Professorkueche/Professorkueche22.jpg', './Bilder/Professorkueche/Professorkueche23.jpg', './Bilder/Professorkueche/Professorkueche24.jpg', './Bilder/Professorkueche/Professorkueche25.jpg', './Bilder/Teekueche/Teekueche20.jpg', './Bilder/Teekueche/Teekueche21.jpg', './Bilder/Teekueche/Teekueche22.jpg', './Bilder/Teekueche/Teekueche23.jpg', './Bilder/Teekueche/Teekueche24.jpg', './Bilder/Teekueche/Teekueche25.jpg']


Hallo, ich bin ein Beispielskript, das die Pfade zu Trainings- und Testbildern enthält.


## Teilaufgabe b): Verteilung RGB

## Teilaufgabe c): Training und Test mit Random Forest 

## Teilaufgabe d): Hyperparameteroptimierung

## Teilaufgabe e): Dimensionalitätsreduktion